# **ORTHORECTIFICATION**

This notebook requires
- file odm_orthophoto.tif from 
datasets\project\odm_orthophoto
in the working directory
- the undistorted images from 
datasets\project\opensfm\undistorted\images
in the directory /undistorted
- orthorectified images from
datasets\project\orthorectified
in the directory /orthorectified

In [ ]:
import numpy as np
import gdal
import pandas as pd
from osgeo import osr, ogr, gdal
from matplotlib import pyplot as plt
import cv2
import os
from google.colab.patches import cv2_imshow
from IPython.display import clear_output
from tqdm import tqdm as tq

current_dir = os.getcwd()

In [ ]:
def from_px_to_lon_lat(ds, x_px, y_px):

  print('converting pixel:', x_px,y_px)

  col, row, band = ds.RasterXSize, ds.RasterYSize, ds.RasterCount
  #print(col, row, band)

  xoff, a, b, yoff, d, e = ds.GetGeoTransform()
  #print(xoff, a, b, yoff, d, e)

  # details about the params: GDAL affine transform parameters
  # xoff,yoff = left corner 
  # a,e = weight,height of pixels
  # b,d = rotation of the image (zero if image is north up)

  def pixel2coord(x, y):
      """Returns global coordinates from coordinates x,y of the pixel"""
      xp = a * x + b * y + xoff
      yp = d * x + e * y + yoff
      return(xp, yp)

  x,y = pixel2coord(x_px,y_px)
  #print(x, y)

  #These global coordinates are in a projected coordinated system, which is a representation of the spheroidal earth's surface, but flattened and distorted onto a plane.
  #To convert these into latitude and longitude, we need to convert these coordinates into geographic coordinate system.


  # get the existing coordinate system
  old_cs= osr.SpatialReference()
  old_cs.ImportFromWkt(ds.GetProjectionRef())

  # create the new coordinate system
  wgs84_wkt = """
  GEOGCS["WGS 84",
      DATUM["WGS_1984",
          SPHEROID["WGS 84",6378137,298.257223563,
              AUTHORITY["EPSG","7030"]],
          AUTHORITY["EPSG","6326"]],
      PRIMEM["Greenwich",0,
          AUTHORITY["EPSG","8901"]],
      UNIT["degree",0.01745329251994328,
          AUTHORITY["EPSG","9122"]],
      AUTHORITY["EPSG","4326"]]"""
  new_cs = osr.SpatialReference()
  new_cs.ImportFromWkt(wgs84_wkt)

  # create a transform object to convert between coordinate systems
  transform = osr.CoordinateTransformation(old_cs,new_cs)

  # converting into geographic coordinate system
  lonx, latx, z = transform.TransformPoint(x,y)

  print('corresponding longitude and latitude:',latx, lonx)
  return (latx, lonx, z)

In [ ]:
def from_lon_lat_to_px(ds, lon, lat):

  print('converting longitude and latitude:', lon, lat)

  width = ds.RasterXSize
  height = ds.RasterYSize
  gt = ds.GetGeoTransform()
  gp = ds.GetProjection()
  data = np.array(ds.ReadAsArray())

  def world_to_pixel(geo_matrix, x, y):
      """
      Uses a gdal geomatrix (gdal.GetGeoTransform()) to calculate
      the pixel location of a geospatial coordinate
      """
      ul_x= geo_matrix[0]
      ul_y = geo_matrix[3]
      x_dist = geo_matrix[1]
      y_dist = geo_matrix[5]
      pixel = int((x - ul_x) / x_dist)
      line = -int((ul_y - y) / y_dist)
      return pixel, line

  # Extract target reference from the tiff file
  target = osr.SpatialReference(wkt=ds.GetProjection())

  source = osr.SpatialReference()
  source.ImportFromEPSG(4326)

  transform = osr.CoordinateTransformation(source, target)

  point = ogr.Geometry(ogr.wkbPoint)
  point.AddPoint(lat, lon) 
  point.Transform(transform)

  x, y = world_to_pixel(ds.GetGeoTransform(), point.GetX(), point.GetY())
  print('corresponding pixel is:',x, y)
  return x,y


In [ ]:
files = []
for file in os.listdir(current_dir+'/orthorectified/'):
    if file.endswith(".tif"):
        files.append(current_dir+'/orthorectified/' + file)
files.sort()

In [ ]:
filens = []
for filee in files:
  filens.append(filee.replace(current_dir + '/orthorectified/', ''))

In [ ]:
centers = []

map_filename = current_dir + '/odm_orthophoto.tif'

ds_map = gdal.Open(map_filename)
img_map = np.array(cv2.imread(map_filename,0))

for filee in files:
  filename = filee

  ds = gdal.Open(filename)
  img = np.array(cv2.imread(filename,0))

  print(img.shape[1]/2, img.shape[0]/2)
  
  x, y = img.shape[1]/2, img.shape[0]/2

  try:
    lon, lat, _ = from_px_to_lon_lat(ds, x, y)

    x2, y2 = from_lon_lat_to_px(ds_map, lon, lat)

    centers.append((x2,y2))
  except:
    print('JUMPED')
    centers.append((0,0))

xs = np.array(centers)[:][:,0]
ys = np.array(centers)[:][:,1]
clear_output()

**SHOTS CENTERS**

In [ ]:
plt.figure(figsize=(15,15))
plt.imshow(img_map, cmap = 'gray')
plt.scatter([xs], [ys], marker="x", color="red", s=250)
plt.show()

In [ ]:
def warp_to_tiff(filen, h_dict):
  
  img1 = cv2.rotate(cv2.imread(current_dir + '/undistorted/'+ filen), cv2.ROTATE_90_COUNTERCLOCKWISE)
  img2 = cv2.imread(current_dir + '/orthorectified/' + filen)
  
  orb = cv2.ORB_create(nfeatures=500)
  kp1, des1 = orb.detectAndCompute(img1, None)
  kp2, des2 = orb.detectAndCompute(img2, None)
  
  ## match descriptors and sort them in the order of their distance
  bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
  matches = bf.match(des1, des2)
  dmatches = sorted(matches, key = lambda x:x.distance)

  ## extract the matched keypoints
  src_pts  = np.float32([kp1[m.queryIdx].pt for m in dmatches]).reshape(-1,1,2)
  dst_pts  = np.float32([kp2[m.trainIdx].pt for m in dmatches]).reshape(-1,1,2)

  ## find homography matrix and do perspective transform
  h, status = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC,5.0)

  
  im_dst = cv2.warpPerspective(img1, h, (img2.shape[:2][1], img2.shape[:2][0]))

  match_img = cv2.drawMatches(img1, kp1, img2, kp2, matches[:50], None)
  
  #cv2_imshow(img2)
  #cv2_imshow(im_dst)
  h_dict[filen] = h
  cv2.imwrite(current_dir + '/orthorectified_original/' + filen, im_dst)
  return h_dict, im_dst

def remap_point(xx,yy,h):
  p = (xx, yy) # your original point
  px = (h[0][0]*p[0] + h[0][1]*p[1] + h[0][2]) / ((h[2][0]*p[0] + h[2][1]*p[1] + h[2][2]))
  py = (h[1][0]*p[0] + h[1][1]*p[1] + h[1][2]) / ((h[2][0]*p[0] + h[2][1]*p[1] + h[2][2]))
  p_after = (int(px), int(py)) # after transformation
  return p_after

**TEST**

In [ ]:
h_dict = {}
for filen in tq(filens):
  h_dict, _ = warp_to_tiff(filen, h_dict)

100%|██████████| 100/100 [00:13<00:00,  7.52it/s]


In [ ]:
idx = 90  #5 50

filen = filens[idx] 
img1 = cv2.rotate(cv2.imread(current_dir + '/undistorted/'+ filen), cv2.ROTATE_90_COUNTERCLOCKWISE)

x,y = 384, 265 #img1.shape[1]/2, img1.shape[0]/2

plt.figure(figsize=(15,15))
plt.imshow(img1, cmap = 'gray')
plt.scatter([x], [y], marker="x", color="red", s=250)
plt.show()

In [ ]:
p_after = remap_point(x,y,h_dict[filen])

img2 = cv2.imread(current_dir + '/orthorectified/'+ filen)

ds_picture = gdal.Open(current_dir + '/orthorectified/'+ filen)


xx,yy = p_after[0], p_after[1]

plt.figure(figsize=(15,15))
plt.imshow(img2, cmap = 'gray')

plt.scatter([xx], [yy], marker="x", color="red", s=250)
plt.show()

lon, lat, _ = from_px_to_lon_lat(ds_picture, xx, yy)

In [ ]:
map_filename = current_dir + '/odm_orthophoto.tif'
ds_map = gdal.Open(map_filename)
img_map = np.array(cv2.imread(map_filename,0))
x_map, y_map = from_lon_lat_to_px(ds_map, lon, lat)

plt.figure(figsize=(40,40))
plt.imshow(img_map, cmap = 'gray')

plt.scatter([x_map], [y_map], marker="x", color="red", s=250)
plt.show()